In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from MyEarlyStopping import MyEarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

## Training dataset (40%)

In [3]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [4]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)

In [5]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
subject = 3
train_image = 10 #10:20%, 20: 40%, 30:60%

In [7]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.], dtype=float32)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)


In [10]:
ALL_Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
ALL_Val_Inputs = np.array(ALL_Val_Inputs)

In [11]:
select_indexs_val = participant_class==subject
Val_Inputs = ALL_Val_Inputs[select_indexs_val]
Val_command_class = Val_command_class[select_indexs_val]
sum(Val_command_class)

array([11., 11., 13., 11., 14., 11., 11., 10., 11., 12.], dtype=float32)

## Test dataset (20%)

In [12]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [13]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)

In [14]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [15]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

In [16]:
# sum(Test_command_class_2)

## Speaker 6 Test dataset 

In [17]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [18]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [19]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [20]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [21]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [22]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [23]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [24]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [25]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [26]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Command_Acc_p15','Command_Acc_p1','Command_Acc_p2',
#                                      'Command_Acc_p3','Command_Acc_p4','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_single_model.csv')
# Perfomance
Perfomance                                                        

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
5,Subjet_1,14.131006,0.5672,0.9279,0.2870,0.4696,0.5780,0.57,0.25,0.328,0.2673,10
6,Subjet_1,13.651466,0.5930,0.9459,0.2685,0.5130,0.6239,0.61,0.43,0.440,0.2673,10
7,Subjet_1,12.100064,0.5727,0.9279,0.2870,0.4957,0.5413,0.61,0.41,0.384,0.2574,10
8,Subjet_1,13.176442,0.6151,0.9459,0.3611,0.4957,0.6147,0.66,0.41,0.408,0.2871,10
9,Subjet_1,13.929373,0.6446,0.9369,0.4722,0.5391,0.6147,0.66,0.48,0.408,0.2673,10


# Create model 

In [27]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Single model （size:10）

In [28]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_3",'Size':'10','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_3_model_size10_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_3_size10_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_3_size10_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
2/2 [==============================] - 2s 867ms/step - loss: 2.5328 - accuracy: 0.0700 - val_loss: 2.2262 - val_accuracy: 0.3565
Epoch 2/10
2/2 [==============================] - 1s 347ms/step - loss: 2.1568 - accuracy: 0.4600 - val_loss: 2.0583 - val_accuracy: 0.6000
Epoch 3/10
2/2 [==============================] - 1s 306ms/step - loss: 1.9921 - accuracy: 0.7000 - val_loss: 1.9600 - val_accuracy: 0.7217
Epoch 4/10
2/2 [==============================] - 1s 300ms/step - loss: 1.8936 - accuracy: 0.8400 - val_loss: 1.8906 - val_accuracy: 0.8000
Epoch 5/10
2/2 [==============================] - 1s 302ms/step - loss: 1.8288 - accuracy: 0.8600 - val_loss: 1.8376 - val_accuracy: 0.8174
Epoch 6/10
2/2 [==============================] - 1s 303ms/step - loss: 1.7802 - accuracy: 0.8700 - val_loss: 1.7954 - val_accuracy: 0.8261
Epoch 7/10
2/2 [==============================] - 1s 319ms/step - loss: 1.7339 - accuracy: 0.8900 - val_loss: 1.7516 - val_accuracy: 0.8348
Epoch 8/10
2/2 [====

2/2 [==============================] - 0s 103ms/step - loss: 1.6612 - accuracy: 0.9000 - val_loss: 1.6775 - val_accuracy: 0.8783
Epoch 1/10
2/2 [==============================] - 1s 321ms/step - loss: 1.6295 - accuracy: 0.9100 - val_loss: 1.6542 - val_accuracy: 0.9043
Epoch 2/10
2/2 [==============================] - 1s 311ms/step - loss: 1.6034 - accuracy: 0.9600 - val_loss: 1.6294 - val_accuracy: 0.9739
Epoch 3/10
2/2 [==============================] - 1s 308ms/step - loss: 1.5784 - accuracy: 0.9900 - val_loss: 1.6076 - val_accuracy: 0.9913
Epoch 4/10
2/2 [==============================] - 0s 109ms/step - loss: 1.5571 - accuracy: 0.9900 - val_loss: 1.5871 - val_accuracy: 0.9826
Epoch 5/10
2/2 [==============================] - 0s 98ms/step - loss: 1.5331 - accuracy: 0.9900 - val_loss: 1.5618 - val_accuracy: 0.9826
Epoch 6/10
2/2 [==============================] - 0s 107ms/step - loss: 1.5063 - accuracy: 0.9900 - val_loss: 1.5396 - val_accuracy: 0.9565
Epoch 7/10
2/2 [================

2/2 [==============================] - 0s 98ms/step - loss: 1.3756 - accuracy: 0.9900 - val_loss: 1.4086 - val_accuracy: 0.9217
Epoch 1/10
2/2 [==============================] - 1s 289ms/step - loss: 1.3340 - accuracy: 0.9900 - val_loss: 1.3712 - val_accuracy: 0.9391
Epoch 2/10
2/2 [==============================] - 0s 98ms/step - loss: 1.2972 - accuracy: 0.9900 - val_loss: 1.3432 - val_accuracy: 0.9217
Epoch 3/10
2/2 [==============================] - 0s 100ms/step - loss: 1.2650 - accuracy: 0.9900 - val_loss: 1.3146 - val_accuracy: 0.9217
Epoch 4/10
2/2 [==============================] - 0s 100ms/step - loss: 1.2329 - accuracy: 1.0000 - val_loss: 1.2849 - val_accuracy: 0.9217
Epoch 5/10
2/2 [==============================] - 0s 102ms/step - loss: 1.2011 - accuracy: 1.0000 - val_loss: 1.2569 - val_accuracy: 0.9391
Epoch 6/10
2/2 [==============================] - 1s 296ms/step - loss: 1.1708 - accuracy: 1.0000 - val_loss: 1.2253 - val_accuracy: 0.9478
Epoch 7/10
2/2 [=================

2/2 [==============================] - 0s 105ms/step - loss: 1.4565 - accuracy: 0.9900 - val_loss: 1.4919 - val_accuracy: 0.9391
Epoch 1/10
2/2 [==============================] - 1s 320ms/step - loss: 1.4160 - accuracy: 1.0000 - val_loss: 1.4631 - val_accuracy: 0.9478
Epoch 2/10
2/2 [==============================] - 0s 110ms/step - loss: 1.3838 - accuracy: 1.0000 - val_loss: 1.4266 - val_accuracy: 0.9478
Epoch 3/10
2/2 [==============================] - 0s 101ms/step - loss: 1.3451 - accuracy: 1.0000 - val_loss: 1.3893 - val_accuracy: 0.9478
Epoch 4/10
2/2 [==============================] - 0s 119ms/step - loss: 1.3109 - accuracy: 1.0000 - val_loss: 1.3568 - val_accuracy: 0.9391
Epoch 5/10
2/2 [==============================] - 1s 328ms/step - loss: 1.2772 - accuracy: 1.0000 - val_loss: 1.3231 - val_accuracy: 0.9565
Epoch 6/10
2/2 [==============================] - 1s 294ms/step - loss: 1.2436 - accuracy: 1.0000 - val_loss: 1.2915 - val_accuracy: 0.9739
Epoch 7/10
2/2 [===============

In [29]:
resnet_model = tf.keras.models.load_model('Initial_subject_3_model_size10_0608.h5')
predictions = resnet_model.predict(Test_Inputs_3)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_3, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_3, axis=1)),4)
print(acc_c)

1.0


In [30]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
65,Subjet_3,13.904538,0.6409,0.4865,0.3981,1.0000,0.6789,0.62,0.40,0.448,0.3366,10
66,Subjet_3,12.220064,0.6483,0.5045,0.4074,0.9913,0.6789,0.64,0.43,0.440,0.3168,10
67,Subjet_3,14.080573,0.6317,0.5045,0.3889,0.9913,0.6239,0.63,0.36,0.464,0.3069,10
68,Subjet_3,12.665570,0.6390,0.5495,0.3704,1.0000,0.6147,0.64,0.37,0.448,0.2574,10


In [31]:
best_index

9

## Single model （size:20）

In [32]:
subject = 3
train_image = 20 #10:20%, 20: 40%, 30:60%

In [33]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20.], dtype=float32)

In [34]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_3",'Size':'20','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_3_model_size20_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_3_size20_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_3_size20_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
4/4 [==============================] - 2s 384ms/step - loss: 2.3545 - accuracy: 0.2850 - val_loss: 1.9593 - val_accuracy: 0.4957
Epoch 2/10
4/4 [==============================] - 1s 182ms/step - loss: 1.8965 - accuracy: 0.5100 - val_loss: 1.7262 - val_accuracy: 0.7652
Epoch 3/10
4/4 [==============================] - 1s 176ms/step - loss: 1.6809 - accuracy: 0.7900 - val_loss: 1.5902 - val_accuracy: 0.8609
Epoch 4/10
4/4 [==============================] - 1s 174ms/step - loss: 1.5586 - accuracy: 0.8950 - val_loss: 1.4810 - val_accuracy: 0.9043
Epoch 5/10
4/4 [==============================] - 1s 174ms/step - loss: 1.4592 - accuracy: 0.9400 - val_loss: 1.4033 - val_accuracy: 0.9652
Epoch 6/10
4/4 [==============================] - 1s 186ms/step - loss: 1.3750 - accuracy: 0.9650 - val_loss: 1.3237 - val_accuracy: 0.9826
Epoch 7/10
4/4 [==============================] - 0s 81ms/step - loss: 1.3065 - accuracy: 0.9900 - val_loss: 1.2632 - val_accuracy: 0.9826
Epoch 8/10
4/4 [=====

Epoch 4/10
4/4 [==============================] - 1s 180ms/step - loss: 1.6325 - accuracy: 0.7300 - val_loss: 1.5724 - val_accuracy: 0.8870
Epoch 5/10
4/4 [==============================] - 1s 174ms/step - loss: 1.5495 - accuracy: 0.8500 - val_loss: 1.4979 - val_accuracy: 0.9391
Epoch 6/10
4/4 [==============================] - 1s 170ms/step - loss: 1.4705 - accuracy: 0.9650 - val_loss: 1.4284 - val_accuracy: 0.9652
Epoch 7/10
4/4 [==============================] - 1s 181ms/step - loss: 1.4037 - accuracy: 0.9900 - val_loss: 1.3648 - val_accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 0s 79ms/step - loss: 1.3403 - accuracy: 0.9900 - val_loss: 1.3070 - val_accuracy: 0.9739
Epoch 9/10
4/4 [==============================] - 0s 88ms/step - loss: 1.2781 - accuracy: 0.9900 - val_loss: 1.2547 - val_accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 0s 84ms/step - loss: 1.2248 - accuracy: 0.9900 - val_loss: 1.1966 - val_accuracy: 1.0000
Epoch 1/10
4/4 [======

Epoch 3/10
4/4 [==============================] - 0s 84ms/step - loss: 1.4229 - accuracy: 0.9900 - val_loss: 1.3882 - val_accuracy: 0.9739
Epoch 4/10
4/4 [==============================] - 0s 83ms/step - loss: 1.3585 - accuracy: 0.9900 - val_loss: 1.3279 - val_accuracy: 0.9565
Epoch 5/10
4/4 [==============================] - 0s 82ms/step - loss: 1.3016 - accuracy: 0.9900 - val_loss: 1.2802 - val_accuracy: 0.9565
Epoch 6/10
4/4 [==============================] - 1s 170ms/step - loss: 1.2522 - accuracy: 0.9900 - val_loss: 1.2296 - val_accuracy: 0.9739
Epoch 1/10
4/4 [==============================] - 1s 369ms/step - loss: 2.5100 - accuracy: 0.1000 - val_loss: 2.2473 - val_accuracy: 0.2696
Epoch 2/10
4/4 [==============================] - 0s 84ms/step - loss: 2.1699 - accuracy: 0.2800 - val_loss: 2.1193 - val_accuracy: 0.2696
Epoch 3/10
4/4 [==============================] - 1s 178ms/step - loss: 2.0511 - accuracy: 0.3600 - val_loss: 2.0035 - val_accuracy: 0.5043
Epoch 4/10
4/4 [========

Epoch 9/10
4/4 [==============================] - 0s 82ms/step - loss: 0.8396 - accuracy: 0.9950 - val_loss: 0.8454 - val_accuracy: 0.9913
Epoch 10/10
4/4 [==============================] - 1s 220ms/step - loss: 0.8019 - accuracy: 0.9950 - val_loss: 0.8081 - val_accuracy: 1.0000


In [35]:
resnet_model = tf.keras.models.load_model('Initial_subject_3_model_size20_0608.h5')
predictions = resnet_model.predict(Test_Inputs_2)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_2, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_2, axis=1)),4)
print(acc_c)

0.4444


In [36]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Subjet_3,15.910140,0.6593,0.4955,0.3611,0.9913,0.7156,0.72,0.32,0.384,0.3267,20
76,Subjet_3,11.082789,0.5838,0.3964,0.3333,0.9739,0.6697,0.52,0.41,0.400,0.2475,20
77,Subjet_3,14.793002,0.6427,0.4775,0.3889,0.9913,0.6697,0.67,0.36,0.448,0.3267,20
78,Subjet_3,14.358196,0.6409,0.5135,0.3611,0.9826,0.6972,0.63,0.40,0.408,0.2475,20


In [37]:
best_index

0

## Single model （size:30）

In [38]:
subject = 3
train_image = 30 #10:20%, 20: 40%, 30:60%

In [39]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30.], dtype=float32)

In [40]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_3",'Size':'30','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_3_model_size30_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_3_size30_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_3_size30_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
5/5 [==============================] - 2s 332ms/step - loss: 2.1817 - accuracy: 0.2433 - val_loss: 1.8347 - val_accuracy: 0.6696
Epoch 2/10
5/5 [==============================] - 1s 191ms/step - loss: 1.7330 - accuracy: 0.8800 - val_loss: 1.6100 - val_accuracy: 0.9739
Epoch 3/10
5/5 [==============================] - 0s 88ms/step - loss: 1.5523 - accuracy: 0.9867 - val_loss: 1.4779 - val_accuracy: 0.9652
Epoch 4/10
5/5 [==============================] - 0s 88ms/step - loss: 1.4353 - accuracy: 0.9900 - val_loss: 1.3795 - val_accuracy: 0.9739
Epoch 5/10
5/5 [==============================] - 0s 86ms/step - loss: 1.3419 - accuracy: 0.9867 - val_loss: 1.2988 - val_accuracy: 0.9652
Epoch 6/10
5/5 [==============================] - 1s 169ms/step - loss: 1.2599 - accuracy: 0.9933 - val_loss: 1.2226 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 1s 112ms/step - loss: 1.1842 - accuracy: 0.9933 - val_loss: 1.1496 - val_accuracy: 1.0000
Epoch 8/10
5/5 [=======

Epoch 10/10
5/5 [==============================] - 0s 85ms/step - loss: 1.2502 - accuracy: 0.9733 - val_loss: 1.2238 - val_accuracy: 0.9565
Epoch 1/10
5/5 [==============================] - 1s 157ms/step - loss: 1.1960 - accuracy: 0.9800 - val_loss: 1.1672 - val_accuracy: 0.9652
Epoch 2/10
5/5 [==============================] - 0s 90ms/step - loss: 1.1382 - accuracy: 0.9967 - val_loss: 1.1110 - val_accuracy: 0.9652
Epoch 3/10
5/5 [==============================] - 1s 164ms/step - loss: 1.0829 - accuracy: 0.9967 - val_loss: 1.0596 - val_accuracy: 0.9826
Epoch 4/10
5/5 [==============================] - 1s 162ms/step - loss: 1.0307 - accuracy: 0.9967 - val_loss: 1.0070 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 98ms/step - loss: 0.9812 - accuracy: 0.9967 - val_loss: 0.9593 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 0s 88ms/step - loss: 0.9320 - accuracy: 0.9967 - val_loss: 0.9149 - val_accuracy: 0.9913
Epoch 7/10
5/5 [=======

Epoch 1/10
5/5 [==============================] - 1s 289ms/step - loss: 2.2539 - accuracy: 0.2433 - val_loss: 1.9319 - val_accuracy: 0.6000
Epoch 2/10
5/5 [==============================] - 1s 165ms/step - loss: 1.8651 - accuracy: 0.6867 - val_loss: 1.7577 - val_accuracy: 0.7652
Epoch 3/10
5/5 [==============================] - 1s 162ms/step - loss: 1.7085 - accuracy: 0.8167 - val_loss: 1.6254 - val_accuracy: 0.9652
Epoch 4/10
5/5 [==============================] - 0s 90ms/step - loss: 1.5948 - accuracy: 0.9833 - val_loss: 1.5470 - val_accuracy: 0.9565
Epoch 5/10
5/5 [==============================] - 0s 89ms/step - loss: 1.5111 - accuracy: 0.9867 - val_loss: 1.4670 - val_accuracy: 0.9652
Epoch 6/10
5/5 [==============================] - 1s 164ms/step - loss: 1.4297 - accuracy: 0.9867 - val_loss: 1.3978 - val_accuracy: 0.9739
Epoch 7/10
5/5 [==============================] - 1s 218ms/step - loss: 1.3568 - accuracy: 0.9967 - val_loss: 1.3232 - val_accuracy: 0.9913
Epoch 8/10
5/5 [======

In [41]:
resnet_model = tf.keras.models.load_model('Initial_subject_3_model_size30_0608.h5')
predictions = resnet_model.predict(Test_Inputs_3)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_3, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_3, axis=1)),4)
print(acc_c)

0.9913


In [42]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
85,Subjet_3,15.712269,0.6427,0.4865,0.3704,0.9913,0.6972,0.65,0.31,0.408,0.3168,30
86,Subjet_3,15.409171,0.6519,0.5315,0.3704,0.9913,0.6972,0.65,0.35,0.408,0.2277,30
87,Subjet_3,14.525968,0.6427,0.4865,0.3519,0.9913,0.6881,0.68,0.37,0.312,0.2574,30
88,Subjet_3,17.149453,0.6280,0.5225,0.3333,0.9913,0.6422,0.63,0.34,0.424,0.2475,30


In [43]:
best_index

2

In [44]:
Perfomance.to_csv('Performance_0608_single_model.csv',index=False)